In [1]:
#import all package function from a local file
from custom_python_package import *

In [7]:
#define the naming used for each file, be it input, output or intermediary files. Used when running the pipeline 
#locally but should be replaced by the correct datasets in the Databricks pipeline

########################
#To be changed following country-specific file name
era5_file_name = 'era5-monthly-seasonal-forecast-total-precipitation-1981-2023-eth'
ecmwf_file_name = 'ecmwf-monthly-seasonalforecast-1981-2023-eth'
admin_file_name = 'eth_admbnda_adm1_csa_bofedb_2021.shp'
admin_code_label = 'ADM1_PCODE'
########################

#input datasets
era5_raw_data_file_path = './data/input_data/' + era5_file_name + '.grib'
ecmwf_raw_data_file_path = './data/input_data/' + ecmwf_file_name + '.grib'
admin_boundary_file_path = './data/input_data/admin_boundary/' + admin_file_name


#ECMWF intermediary datasets
ecmwf_processed_pixel_file_path = './data/temp_data/' + ecmwf_file_name + '-processed-pixel' + '.parquet.gzip'
ecmwf_processed_adm_file_path = './data/temp_data/' + ecmwf_file_name + '-processed-adm' + '.parquet.gzip'
ecmwf_bias_corr_pixel_file_path = './data/temp_data/' + ecmwf_file_name + '-bias-corrected-pixel' + '.parquet.gzip'
ecmwf_bias_corr_adm_file_path = './data/temp_data/' + ecmwf_file_name + '-bias-corrected-adm' + '.parquet.gzip'
ref_grid_file_path = './data/temp_data/' + ecmwf_file_name + '-reference-grid' + '.parquet.gzip'


#ERA5 intermediary/export datasets
era5_processed_pixel_file_path = './data/output_data/' + era5_file_name + '-processed-pixel' + '.parquet.gzip'
era5_processed_adm_file_path = './data/output_data/' + era5_file_name + '-processed-adm' + '.parquet.gzip'

#ECMWF export datasets
ecmwf_quantile_pixel_file_path = './data/output_data/' + ecmwf_file_name + '-quantile-pixel' + '.parquet.gzip'
ecmwf_quantile_bias_corr_pixel_file_path = './data/output_data/' + ecmwf_file_name + '-quantile-bias-corrected-pixel' + '.parquet.gzip'
ecmwf_quantile_adm_file_path = './data/output_data/' + ecmwf_file_name + '-quantile-adm' + '.parquet.gzip'
ecmwf_quantile_bias_corr_adm_file_path = './data/output_data/' + ecmwf_file_name + '-quantile-bias-corrected-adm' + '.parquet.gzip'


In [3]:
#Converts ECMWF grib file into a dataframe, start processing the format and exports it to a parquet file
pre_process_ecmwf_data(ecmwf_raw_data_file_path, admin_boundary_file_path, ref_grid_file_path, ecmwf_processed_pixel_file_path, ecmwf_processed_adm_file_path, admin_code_label)


processing ecmwf data...
0/50


/home/daniel/Desktop/MapAction/CHD/test_env/custom_python_package.py:93: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid_df['geometry'] = grid_df['geometry'].buffer(0.5, cap_style = 3)


10/50
20/50
30/50
40/50
50/50
data exported


In [8]:
#Converts ERA5 grib file into a dataframe, start processing the format and exports it to a parquet file
pre_process_era5_data(era5_raw_data_file_path, ecmwf_raw_data_file_path, ref_grid_file_path, era5_processed_pixel_file_path, era5_processed_adm_file_path)


/home/daniel/anaconda3/envs/chd-project/lib/python3.12/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


OSError: Cannot save file into a non-existent directory: 'data/output_data_bis'

In [5]:
#Correct the bias between ECMWF and ERA5 values (for every location and month). Executes it twice, one at the 
#grid point level (pixel) and one at the admin boundary level
ecmwf_bias_correction(ecmwf_processed_pixel_file_path, era5_processed_pixel_file_path, ecmwf_bias_corr_pixel_file_path)
ecmwf_bias_correction(ecmwf_processed_adm_file_path, era5_processed_adm_file_path, ecmwf_bias_corr_adm_file_path)


In [6]:
#Compute quantile probabilities based on ERA5 climatology and ECMWF ensemble models. Executes it four times, following
#the geospatial unit (pixel or administrative boundary level) and the bias correction status (before and after bias correction)
compute_quantile_probability(ecmwf_processed_pixel_file_path, era5_processed_pixel_file_path, ecmwf_quantile_pixel_file_path)
compute_quantile_probability(ecmwf_bias_corr_pixel_file_path, era5_processed_pixel_file_path, ecmwf_quantile_bias_corr_pixel_file_path)
compute_quantile_probability(ecmwf_processed_adm_file_path, era5_processed_adm_file_path, ecmwf_quantile_adm_file_path)
compute_quantile_probability(ecmwf_bias_corr_adm_file_path, era5_processed_adm_file_path, ecmwf_quantile_bias_corr_adm_file_path)
